In [1]:
import numpy as np
import random as rnd
from numpy import array as ar
from math import floor
from IPython.display import clear_output

In [2]:
PLAYER_1 = 1
PLAYER_2 = 2

In [13]:
def rewardForState(board, player):

    for i in range(3):
        if board[i][0] == board[i][1] and board[i][2] == board[i][1]:
            return (5 * (1 if board[i][1] == player else -1) if board[i][1] != 0 else 0)
        
        elif board[0][i] == board[1][i] and board[2][i] == board[1][i]:
            return (5 * (1 if board[1][i] == player else -1) if board[1][i] != 0 else 0)
        
    if board[0][0] == board[1][1] and board[2][2] == board[1][1]:
        return (5 * (1 if board[1][1] == player else -1) if board[1][1] != 0 else 0)
    
    if board[0][2] == board[1][1] and board[2][0] == board[1][1]:
        return (5 * (1 if board[1][1] == player else -1) if board[1][1] != 0 else 0)
    return 0

In [5]:
def actionReward(board, player, action):
    if board[floor(action / 3)][action % 3] != 0:
        return -100
    else:
        board[floor(action / 3)][action % 3] += player
        return rewardForState(board, player)

In [6]:
def val(board):
    val = 0
    for i in range(9):
        val += board[floor(i / 3)][i % 3] * 3**i
    return int(val)

In [7]:
def doAction(player, action, board):
    reward = actionReward(board, player, action)
    done = False
    if (reward != 0 and reward != -100) or np.prod(board) != 0:
        done = True
    return (reward, done, val(board))

In [21]:
q_table = np.zeros([2, 3**9, 9])

for i in range(100000):
    board = np.zeros((3, 3))
    
    alpha = 0.9
    gamma = 0.6
    epsilon = 0.9
    
    done = False
    
    player = 1
    state = 0
    prev_state = 0
    prev_action = 0
    
    while not done:
        epsilon = 0.9 - 0.9 * i / 100000
        if rnd.uniform(0, 1) < epsilon:
            action = rnd.randint(0, 8)
        else:
            action = np.argmax(q_table[player - 1, state])
#             print("Q _ Actions  \n", q_table[player - 1, state], end = "\n\n")
        
        reward, done, next_state = doAction(player, action, board)
        
        spect_player = 2 if player == 1 else 1
        old_val_player = q_table[player - 1, state, action]
        old_val_splayer = q_table[spect_player - 1, prev_state, prev_action]
        
        next_max = np.argmax(q_table[player - 1, next_state])
#         if i % 100:
#             print("NEXT_MAX ", next_max)
#             print("next _ state", next_state)
        q_table[player - 1, state, action] = old_val_player * alpha + (reward + gamma * q_table[player - 1, state, next_max]) * (1 - alpha)
        
        if reward > 0:
            q_table[spect_player - 1, prev_state, prev_action] = -reward
            
#         print(board , " Player :", player ,"Reward : ", reward, "Next Max : ", next_max, "Q : ", q_table[player - 1, state, action], end='\n\n\n')
        if done and i > 99990:
            print(reward, " Player = ", player)
            print(board)
        player = 1 if player == 2 else 2
        
        prev_state = state
        prev_action = action
        
        state = next_state

0  Player =  1
[[1. 2. 1.]
 [2. 1. 1.]
 [2. 1. 2.]]
0  Player =  1
[[1. 2. 1.]
 [2. 1. 1.]
 [2. 1. 2.]]
0  Player =  1
[[1. 2. 1.]
 [2. 1. 1.]
 [2. 1. 2.]]
0  Player =  1
[[1. 2. 1.]
 [2. 1. 1.]
 [2. 1. 2.]]
0  Player =  1
[[1. 2. 1.]
 [2. 1. 1.]
 [2. 1. 2.]]
0  Player =  1
[[1. 2. 1.]
 [2. 1. 1.]
 [2. 1. 2.]]
0  Player =  1
[[1. 2. 1.]
 [2. 1. 1.]
 [2. 1. 2.]]
0  Player =  1
[[1. 2. 1.]
 [2. 1. 1.]
 [2. 1. 2.]]
0  Player =  1
[[1. 2. 1.]
 [2. 1. 1.]
 [2. 1. 2.]]


In [27]:
board = np.zeros((3, 3))

In [34]:
doAction(1, 3, board)
print(board)

[[2. 1. 1.]
 [1. 1. 0.]
 [2. 2. 0.]]


In [35]:
doAction(2, np.argmax(q_table[player - 1, val(board)]), board)
print(board)

[[2. 1. 1.]
 [1. 1. 2.]
 [2. 2. 0.]]
